In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('analysts_survey_1.1.csv')

pd.set_option('display.max_rows', None)

df.columns

Index(['analyst_id', 'gender', 'age', 'data_analyzing', 'analyst_type',
       'data_science', 'living_area', 'office_area', 'job_percentage',
       'sallary', 'min_sallary Range', 'max_sallary Range', 'avg_sallary',
       'equity', 'experience_years', 'manager', 'school_degree',
       'course_taken', 'course_name', 'visualization_tool', 'using_sql',
       'required_statistic_knowledge', 'predictive_models',
       'wages_decrease_covid19', 'fired_covid19', 'company', 'startup_size'],
      dtype='object')

In [10]:
#cleaning 

#drop 
df.dropna(axis=0, how="any", thresh=None, subset=['avg_sallary'], inplace=True)
df['avg_sallary'].isna().sum()

df = df[df['job_percentage'] == 'Full-time']
df['job_percentage'].isna().sum()

df = df[df['data_analyzing'] == 'Yes']
df['data_analyzing'].isna().sum()

df.drop(df[df['company'] == 'IDF'].index, inplace=True)

# drop the 1 na so i can use this field in the regression model
df.dropna(axis=0, how="any", thresh=None, subset=['course_taken'], inplace=True)
df['course_taken'].isna().sum()

df.dropna(axis=0, how="any", thresh=None, subset=['equity'], inplace=True)
df['equity'].isna().sum()

0

In [3]:
#df[['data_science']].describe()

In [4]:
#Adjusting data - changing text values to numbers

df['gender'].replace(['Male', 'Female'], [0, 1], inplace=True)
df['data_science'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['using_sql'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['predictive_models'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['wages_decrease_covid19'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['fired_covid19'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['equity'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['manager'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['course_taken'].replace(['No', 'Yes'], [0, 1], inplace=True)

#required_statistic_knowledge
df['required_statistic_knowledge'].replace(['No', 'Little', 'Medium', 'High'], [0, 1, 2, 3], inplace=True)

# cleaning but anyway - wont be in the model since all values are the same : Yes 
df['data_analyzing'].replace(['Yes'], [1], inplace=True)

In [5]:
#df.dtypes

In [6]:
# df data science only
df_data_science = df[df['data_science'] == 1]
df_data_science

,analyst_id,gender,age,data_analyzing,analyst_type,data_science,living_area,office_area,job_percentage,sallary,...,course_taken,course_name,visualization_tool,using_sql,required_statistic_knowledge,predictive_models,wages_decrease_covid19,fired_covid19,company,startup_size
48,49,0,48,1,Data sceintist,1,Center,Tel Aviv,Full-time,25 - 28,...,1,Naya College,"Power BI, Qlik view \ Qlik sense",1,0,1,0,0,Corporate,NaN
56,57,1,31,1,Data sceintist,1,Tel Aviv,Tel Aviv,Full-time,18 - 20,...,0,Udemy,JMP,0,2,1,0,0,Startup,Early Stage Startup (0-20)
69,70,1,35,1,Data sceintist,1,Haifa,Home,Full-time,35 - 38,...,0,NaN,"Power BI, Don't use",0,3,1,0,0,Startup,Early Stage Startup (0-20)
104,105,0,33,1,Data sceintist,1,Jerusalem,Tel Aviv,Full-time,30 - 33,...,0,NaN,"Tableau, ggplot",1,3,1,0,0,GAMFA,NaN
112,113,0,34,1,Data sceintist,1,Center,Tel Aviv,Full-time,18 - 20,...,0,NaN,Don't use,1,3,1,0,0,Corporate,NaN
170,171,0,34,1,Data sceintist,1,Center,Center,Full-time,40,...,0,NaN,"Tableau, Redash",1,3,1,1,0,Startup,Unkown
202,203,1,27,1,Data sceintist,1,Tel Aviv,Tel Aviv,Full-time,25 - 28,...,0,NaN,Tableau,1,3,1,0,0,GAMFA,NaN
223,224,0,39,1,Data sceintist,1,Center,Center,Full-time,15 - 18,...,1,Naya College,Excel,1,1,1,0,1,Startup,Small Startup (20-50)
224,225,1,25,1,Data sceintist,1,Tel Aviv,Tel Aviv,Full-time,15 - 18,...,0,NaN,Don't use,0,1,1,0,0,Startup,Small Startup (20-50)
259,260,0,40,1,Data sceintist,1,Center,Center,Full-time,30 - 33,...,1,Ofisoft,Pyramid,1,1,1,0,0,Startup,Unkown


In [7]:
#df_data_science['experience_years'].mean() #6.67
#df_data_science['experience_years'].median() #5
#df_data_science['avg_sallary'].mean() # 28.42
#df_data_science['avg_sallary'].median() #29.5

In [8]:
df_data_science[["experience_years", "avg_sallary", "data_science", "visualization_tool", "school_degree", "course_taken", "course_name", "using_sql", "gender", "company" ]]

,experience_years,avg_sallary,data_science,visualization_tool,school_degree,course_taken,course_name,using_sql,gender,company
48,21.0,27.0,1,"Power BI, Qlik view \ Qlik sense",Computer Science,1,Naya College,1,0,Corporate
56,2.0,19.0,1,JMP,Engineering - Other,0,Udemy,0,1,Startup
69,10.0,37.0,1,"Power BI, Don't use",Statistics,0,NaN,0,1,Startup
104,9.0,32.0,1,"Tableau, ggplot",Economics,0,NaN,1,0,GAMFA
112,3.0,19.0,1,Don't use,Economics,0,NaN,1,0,Corporate
170,5.0,40.0,1,"Tableau, Redash",Statistics,0,NaN,1,0,Startup
202,3.0,27.0,1,Tableau,Computer Science,0,NaN,1,1,GAMFA
223,3.0,17.0,1,Excel,Economics,1,Naya College,1,0,Startup
224,0.0,17.0,1,Don't use,Electrical Engineering and Physics,0,NaN,0,1,Startup
259,11.0,32.0,1,Pyramid,Business Administration,1,Ofisoft,1,0,Startup


In [9]:
# data science - experience years 
df_data_science["avg_sallary"].groupby([df_data_science["data_science"], pd.cut(df_data_science["experience_years"], [0,1,3,6,20])]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])


average  median   max       std  count_nonzero
data_science experience_years                                                
1            (0, 1]                NaN     NaN   NaN       NaN            NaN
             (1, 3]               20.5    19.0  27.0  4.434712            4.0
             (3, 6]               38.5    38.5  40.0  2.121320            2.0
             (6, 20]              34.5    34.5  37.0  2.886751            4.0

In [ ]:
# data science - visualization tool
df_data_science["avg_sallary"].groupby([df_data_science["data_science"], df_data_science["visualization_tool"]]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
# data science vs data analysts
#df_data_science["avg_sallary"].groupby([df_data_science["data_science"]]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
# data science school degrees
#df_data_science["avg_sallary"].groupby([df_data_science["data_science"], df_data_science["school_degree"]]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
# data science experience years gender
#df_data_science["avg_sallary"].groupby([df_data_science["data_science"], df_data_science["gender"], pd.cut(df1_ds["experience_years"], [0,1,3,6,20])]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
df_data_science["avg_sallary"].groupby([pd.cut(df_data_science["experience_years"], [0,1,3,6,20]),df_data_science["gender"], df_data_science["data_science"]]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])


In [ ]:
# data science SQL expereience years
#df_data_science["avg_sallary"].groupby([df_data_science["data_science"], df_data_science["using_sql"], pd.cut(df_data_science["experience_years"], [0,1,3,6,20])]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
# data science, company
#df_data_science["avg_sallary"].groupby([df_data_science["data_science"], pd.cut(df_data_science["experience_years"], [0,1,3,6,20]), df_data_science["company"]]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
# school degree for data scientists
#df_data_science["avg_sallary"].groupby([df_data_science["data_science"], df_data_science["school_degree"]]).aggregate([np.average, np.median, max, np.std, np.count_nonzero])

In [ ]:
###df_ds_sql_salary = df.groupby('data_science')[['avg_sallary', 'using_sql']].mean().reset_index()
###df_ds_sql_salary